In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import tqdm
from copy import deepcopy
import collections

In [2]:
class ReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = collections.deque(maxlen=maxlen)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        ind = np.random.choice(len(self.buffer), n, replace=False)
        s, a, r, d, ns = zip(*[self.buffer[i] for i in ind])
        return np.array(s), np.array(a), np.array(r), \
            np.array(d), np.array(ns)
    
    def size(self):
        return len(self.buffer)

In [3]:
class Net(nn.Module):
    def __init__(self, input_dim=4, hid_dim=128, nlayers=2, out_dim=2):
        super(Net, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model.append(nn.Linear(inp, out_dim))
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x)

In [4]:
class DQN:
    def __init__(self, model, optim, gamma=0.9, update_freq=300):
        self.model = model
        self.optim = optim
        self.gamma = gamma
        self.update_freq = update_freq
        self._cnt = 0
        self.model_target = deepcopy(self.model)
        self.model_target.eval()
    
    def learn(self, buffer, batch_size):
        self.model.train()
        
        s, a, r, d, ns = buffer.sample(batch_size)
        s = torch.from_numpy(s).float()
        a = torch.from_numpy(a).long()
        ns = torch.from_numpy(ns).float()
        d = torch.from_numpy(d).float()
        r = torch.from_numpy(r).float()
        
        q_a = self.model(s)[torch.arange(len(a)), a]
        max_target_q = self.model_target(ns).detach().max(dim=1)[0]
        # very important multiply d
        
        target = r + self.gamma * max_target_q * d
        
        loss = F.mse_loss(q_a, target)
        
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()
        self._cnt += 1
        
        if self._cnt % self.update_freq == 0:
            self.model_target.load_state_dict(self.model.state_dict())
    
    def sample_action(self, state, eps):
        self.model.eval()
        state = torch.from_numpy(state)
        with torch.no_grad():
            out = self.model(state)
            action = out.argmax().item()
        
        if np.random.rand() < eps:
            action = np.random.randint(len(out))
        
        return action

In [ ]:
env = gym.make('CartPole-v1')

In [6]:
buffer = ReplayBuffer(maxlen=50000)
lr = 5e-4
gamma = 0.98
batch_size = 32
max_episodes = 1000

n_obs = env.observation_space.shape[0]
n_act = env.action_space.n
model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [7]:
policy = DQN(model, optimizer, gamma, update_freq=20)

In [ ]:
all_score_list = []
for k in range(10):
    score_list = []
    
    buffer = ReplayBuffer(maxlen=50000)
    lr = 5e-4
    gamma = 0.98
    batch_size = 32
    max_episodes = 1000

    n_obs = env.observation_space.shape[0]
    n_act = env.action_space.n
    model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    policy = DQN(model, optimizer, gamma, update_freq=20)
    
    for i in tqdm.tqdm(range(max_episodes)):
        score = 0
        s = env.reset()
        eps = max(0.01, 0.08 - 0.01*i/200)
        done = False
        while not done:
            a = policy.sample_action(s, eps)
            next_s, r, done, info = env.step(a)
            d = 0.0 if done else 1.0
            buffer.put((s, a, r/100, d, next_s))
            s = next_s

            score += r

        if buffer.size() > 200:
            policy.learn(buffer, batch_size)

        score_list.append(score)
    all_score_list.append(score_list)